## Load ImageNet Base Model

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


## Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [2]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [3]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224,224,3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [5]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [7]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [8]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
37/36 [==============================] - 28s 753ms/step - loss: 2.1666 - accuracy: 0.5118 - val_loss: 0.9729 - val_accuracy: 0.7356
Epoch 2/20
37/36 [==============================] - 20s 531ms/step - loss: 0.7954 - accuracy: 0.7614 - val_loss: 0.5207 - val_accuracy: 0.8480
Epoch 3/20
37/36 [==============================] - 20s 529ms/step - loss: 0.4558 - accuracy: 0.8503 - val_loss: 0.4017 - val_accuracy: 0.8693
Epoch 4/20
37/36 [==============================] - 20s 529ms/step - loss: 0.2778 - accuracy: 0.9086 - val_loss: 0.2285 - val_accuracy: 0.9210
Epoch 5/20
37/36 [==============================] - 20s 531ms/step - loss: 0.2003 - accuracy: 0.9255 - val_loss: 0.2924 - val_accuracy: 0.8997
Epoch 6/20
37/36 [==============================] - 20s 528ms/step - loss: 0.1461 - accuracy: 0.9467 - val_loss: 0.1260 - val_accuracy: 0.9483
Epoch 7/20
37/36 [==============================] - 20s 532ms/step - loss: 0.1177 - accuracy: 0.9602 - val_loss: 0.1254 - val_accuracy: 0.9574

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [ ]:
# Unfreeze the base model
base_model.trainable = FIXME

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = FIXME),
              loss = FIXME , metrics = FIXME)

In [ ]:
model.fit(FIXME,
          validation_data=FIXME,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=FIXME)

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [9]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 350ms/step - loss: 0.1215 - accuracy: 0.9696


[0.12153420597314835, 0.9696048498153687]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [10]:
from run_assessment import run_assessment

In [11]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

11/10 [================================] - 4s 345ms/step - loss: 0.1158 - accuracy: 0.9726

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9745.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.
